In [2]:
import json
import os

total_dictionaries = 0
json_file_directory = "./Information Json" # 请替换为实际的目录路径

# 遍历目录下的所有文件
for filename in os.listdir(json_file_directory):
    if filename.endswith(".json"): # 确保只处理JSON文件
        filepath = os.path.join(json_file_directory, filename)
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                data = json.load(f) # 解析JSON文件
                if isinstance(data, list): # 确保解析出来的是一个列表
                    total_dictionaries += len(data) # 累加字典数量
                else:
                    print(f"警告: 文件 '{filename}' 的根元素不是一个列表，跳过统计。")
        except json.JSONDecodeError:
            print(f"错误: 文件 '{filename}' 不是一个有效的JSON文件，跳过。")
        except Exception as e:
            print(f"读取文件 '{filename}' 时发生错误: {e}")

print(f"所有JSON文件中总共有 {total_dictionaries} 个字典。")

警告: 文件 '10.1002-adma.202500310.json' 的根元素不是一个列表，跳过统计。
警告: 文件 '10.1002_cjoc.70149.json' 的根元素不是一个列表，跳过统计。
警告: 文件 '10.1016_j.trac.2023.117409.json' 的根元素不是一个列表，跳过统计。
警告: 文件 '10.1039-d0dt04276e_1747625499254894_huangfu2021.json' 的根元素不是一个列表，跳过统计。
所有JSON文件中总共有 2070 个字典。


In [18]:
import os
import shutil

def match_files_by_processed_doi(doi_file_path, source_dir, target_dir, output_file):
    matched = []
    unmatched = []
    short = []
    # 创建目标目录
    os.makedirs(target_dir, exist_ok=True)



    # 读取 DOI 文件
    with open(doi_file_path, 'r', encoding='utf-8') as f:
        dois = [line.strip() for line in f if line.strip()]

    # 处理每一个 DOI
    for doi in dois:
        # 第一步：用 / 分割，取最后一段
        parts_after_slash = doi.split('/')
        if len(parts_after_slash) < 2:
            unmatched.append(doi)
            continue
        last_part = parts_after_slash[-1]

        # 第二步：对最后一段继续分割，比如取 . 的最后一部分（可选）
        id_to_match = last_part.split('.')[-1].strip()
        # id_to_match = last_part
        if len(id_to_match) < 4:
            short.append(doi)
        else:
            # 尝试匹配文件名（忽略大小写）
            found = False
            # print(id_to_match)
                # 获取所有源文件名（只保留文件）
            all_files = [
                f for f in os.listdir(source_dir)
                if os.path.isfile(os.path.join(source_dir, f))
            ]
            for filename in all_files:
                # print(filename)
                if id_to_match.lower() in filename.lower():
                    
                    # 匹配成功，移动文件
                    src = os.path.join(source_dir, filename)
                    dst = os.path.join(target_dir, filename)
                    shutil.move(src, dst)
                    matched.append(doi)
                    found = True
                    break

            if not found:
                unmatched.append(doi)

    # 写入报告
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("✅ Matched DOIs:\n")
        for d in matched:
            f.write(f"{d}\n")
        f.write("\n❌ Unmatched DOIs:\n")
        for d in unmatched:
            f.write(f"{d}\n")
        f.write("\n❌ short DOIs:\n")
        for d in short:
            f.write(f"{d}\n")

    print(f"共匹配到 {len(matched)} 个 DOI，未匹配 {len(unmatched)} 个。")

# 使用函数
doi_file_path = "./screeningdoi2.txt"
source_dir = "./Information Json"
target_dir = "./ScreeningCard"
output_file = "./report.txt"

match_files_by_processed_doi(doi_file_path, source_dir, target_dir, output_file)

共匹配到 0 个 DOI，未匹配 308 个。
